Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [105]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [106]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [107]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [108]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [109]:
batch_size = 128

n_input = image_size * image_size
n_hidden_1 = 1024

learning_rate = 0.5

graph = tf.Graph()
with graph.as_default():
    
    x = tf.placeholder(tf.float32, [batch_size, n_input])
    y_ = tf.placeholder(tf.float32, [batch_size, num_labels])
    
    beta = tf.placeholder(tf.float32)
    
    weights = {
        'h1' : tf.Variable(tf.truncated_normal([n_input, n_hidden_1])),
        'out' : tf.Variable(tf.truncated_normal([n_hidden_1, num_labels]))
    }
    
    biases = {
        'h1' : tf.Variable(tf.zeros([n_hidden_1])),
        'out' : tf.Variable(tf.zeros([num_labels]))
    }
    
    hidden_1 = tf.matmul(x, weights['h1']) + biases['h1']
    hidden_1 = tf.nn.relu(hidden_1)
    
    out_l = tf.matmul(hidden_1, weights['out']) + biases['out']
    
    reg = beta/2 * (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(biases['h1']) + tf.nn.l2_loss(weights['out'])+ tf.nn.l2_loss(biases['out']))
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out_l, y_) +  reg)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
    
    train_prediction = tf.nn.softmax(out_l)
    
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    
    valid_prediction = tf.nn.softmax(

        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_valid_dataset, weights['h1']) + biases['h1']
            ), weights['out']
        ) + biases['out']
    )
    
    test_prediction = tf.nn.softmax(

        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_test_dataset, weights['h1']) + biases['h1']
            ), weights['out']
        ) + biases['out']
    )
    

In [110]:
def train(num_steps, reg_rate):
    tf.set_random_seed(27)
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print('Initialized')
        for step in range(num_steps):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

            batch_data = train_dataset[offset:(offset+batch_size), :]
            batch_labels = train_labels[offset:(offset+batch_size), :]

            feed_dict = {x:batch_data, y_ : batch_labels, beta: reg_rate}

            _, l, predictions = session.run([optimizer,loss,train_prediction], feed_dict=feed_dict)

            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(
                        valid_prediction.eval(), valid_labels))

        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [111]:
num_steps = 3001
reg_rate = 0.003025 #0.03125 #  

train(num_steps, reg_rate)

Initialized
Minibatch loss at step 0: 866.922607
Minibatch accuracy: 10.2%
Validation accuracy: 33.7%
Minibatch loss at step 500: 232.704849
Minibatch accuracy: 74.2%
Validation accuracy: 78.9%
Minibatch loss at step 1000: 104.344452
Minibatch accuracy: 79.7%
Validation accuracy: 82.1%
Minibatch loss at step 1500: 48.408432
Minibatch accuracy: 89.1%
Validation accuracy: 84.3%
Minibatch loss at step 2000: 22.699335
Minibatch accuracy: 89.1%
Validation accuracy: 86.0%
Minibatch loss at step 2500: 10.881293
Minibatch accuracy: 89.1%
Validation accuracy: 86.8%
Minibatch loss at step 3000: 5.428681
Minibatch accuracy: 88.3%
Validation accuracy: 86.9%
Test accuracy: 93.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [112]:
num_steps = 4
reg_rate = 0.003025 #  0.03125

train(num_steps, reg_rate)

Initialized
Minibatch loss at step 0: 801.417725
Minibatch accuracy: 10.9%
Validation accuracy: 29.2%
Test accuracy: 54.1%


#### Response
Final Test Accuracy reduces when training data with just a few batches.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [113]:
tf.set_random_seed(27)
batch_size = 128

n_input = image_size * image_size
n_hidden_1 = 1024

learning_rate = 0.5

graph = tf.Graph()
with graph.as_default():
    
    x = tf.placeholder(tf.float32, [batch_size, n_input])
    y_ = tf.placeholder(tf.float32, [batch_size, num_labels])
    
    beta = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder("float")
    
    weights = {
        'h1' : tf.Variable(tf.truncated_normal([n_input, n_hidden_1])),
        'out' : tf.Variable(tf.truncated_normal([n_hidden_1, num_labels]))
    }
    
    biases = {
        'h1' : tf.Variable(tf.zeros([n_hidden_1])),
        'out' : tf.Variable(tf.zeros([num_labels]))
    }
    
    hidden_1 = tf.matmul(x, weights['h1']) + biases['h1']
    hidden_l = tf.nn.dropout(tf.nn.relu(hidden_1), keep_prob)
    
    out_l = tf.matmul(hidden_1, weights['out']) + biases['out']
    
    reg = beta/2 * (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(biases['h1']) + tf.nn.l2_loss(weights['out'])+ tf.nn.l2_loss(biases['out']))
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out_l, y_) +  reg)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
    
    train_prediction = tf.nn.softmax(out_l)
    
    tf_test_dataset = tf.constant(test_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    
    valid_prediction = tf.nn.softmax(

        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_valid_dataset, weights['h1']) + biases['h1']
            ), weights['out']
        ) + biases['out']
    )
    
    test_prediction = tf.nn.softmax(

        tf.matmul(
            tf.nn.relu(
                tf.matmul(tf_test_dataset, weights['h1']) + biases['h1']
            ), weights['out']
        ) + biases['out']
    )    

In [114]:
# train normal
num_steps = 3001
reg_rate = 0.003025 #  0.03125

train(num_steps, reg_rate)

Initialized
Minibatch loss at step 0: 950.871826
Minibatch accuracy: 10.2%
Validation accuracy: 37.9%
Minibatch loss at step 500: 230.246597
Minibatch accuracy: 67.2%
Validation accuracy: 63.6%
Minibatch loss at step 1000: 105.724899
Minibatch accuracy: 72.7%
Validation accuracy: 69.2%
Minibatch loss at step 1500: 46.677925
Minibatch accuracy: 79.7%
Validation accuracy: 67.1%
Minibatch loss at step 2000: 21.839909
Minibatch accuracy: 78.1%
Validation accuracy: 72.9%
Minibatch loss at step 2500: 10.486496
Minibatch accuracy: 73.4%
Validation accuracy: 74.7%
Minibatch loss at step 3000: 5.217443
Minibatch accuracy: 80.5%
Validation accuracy: 78.8%
Test accuracy: 86.0%


In [115]:
# train with small sample
num_steps = 4
reg_rate = 0.003025
train(num_steps, reg_rate)

Initialized
Minibatch loss at step 0: 984.442017
Minibatch accuracy: 4.7%
Validation accuracy: 42.3%
Test accuracy: 55.2%


#### Result
Test accuracy increases in about 20%.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [116]:
def calc_prediction(dataset, weights, biases):
    
    hidden_1 = tf.matmul(dataset, weights['h1']) + biases['h1']
    hidden_l = tf.nn.relu(hidden_1)
    
    hidden_2 = tf.matmul(hidden_1, weights['h2']) + biases['h2']
    hidden_2 = tf.nn.relu(hidden_2)
    
#    hidden_3 = tf.matmul(hidden_2, weights['h3']) + biases['h3']
#    hidden_3 = tf.nn.relu(hidden_3)
    
    out_l = tf.matmul(hidden_2, weights['out']) + biases['out']
    
    prediction = tf.nn.softmax(out_l)
    
    return prediction


batch_size = 128

n_input = image_size * image_size
n_hidden_1 = n_hidden_2 = n_hidden_3 = 1024

graph = tf.Graph()
with graph.as_default():
    
    x = tf.placeholder(tf.float64, [batch_size, n_input])
    y_ = tf.placeholder(tf.float64, [batch_size, num_labels])
    
    beta = tf.placeholder(tf.float64)
    
    weights = {
        'h1' : tf.Variable(tf.truncated_normal([n_input, n_hidden_1],dtype=tf.float64)),
        'h2' : tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2],dtype=tf.float64)),
#        'h3' : tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3],dtype=tf.float64)),        
        'out' : tf.Variable(tf.truncated_normal([n_hidden_2, num_labels],dtype=tf.float64))
    }
    
    biases = {
        'h1' : tf.Variable(tf.zeros([n_hidden_1],tf.float64)),
        'h2' : tf.Variable(tf.zeros([n_hidden_2],tf.float64)),
#        'h3' : tf.Variable(tf.zeros([n_hidden_3],tf.float64)),        
        'out' : tf.Variable(tf.zeros([num_labels],tf.float64))
    }
    
    hidden_1 = tf.matmul(x, weights['h1']) + biases['h1']
    hidden_l = tf.nn.relu(hidden_1)
    
    hidden_2 = tf.matmul(hidden_1, weights['h2']) + biases['h2']
    hidden_2 = tf.nn.relu(hidden_2)
    
#    hidden_3 = tf.matmul(hidden_2, weights['h3']) + biases['h3']
#    hidden_3 = tf.nn.relu(hidden_3)
    
    out_l = tf.matmul(hidden_2, weights['out']) + biases['out']
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out_l, y_) + (
                         beta/2 * (tf.nn.l2_loss(weights['h1']) + tf.nn.l2_loss(biases['h1']) +
                                   tf.nn.l2_loss(weights['h2']) + tf.nn.l2_loss(biases['h2']) +
 #                                  tf.nn.l2_loss(weights['h3']) + tf.nn.l2_loss(biases['h3']) +
                                   tf.nn.l2_loss(weights['out']) + tf.nn.l2_loss(biases['out'])
                         )))
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.003).minimize(loss)
    
    train_prediction = tf.nn.softmax(out_l)
    
    tf_test_dataset = tf.constant(test_dataset, dtype=tf.float64)
    tf_valid_dataset = tf.constant(valid_dataset, dtype=tf.float64)
    
    valid_prediction = calc_prediction(tf_valid_dataset, weights, biases)
    test_prediction = calc_prediction(tf_test_dataset, weights, biases)    

In [117]:
# train normal
num_steps = 3001
reg_rate = 0.0030 #  0.03125

train(num_steps, reg_rate)

Initialized
Minibatch loss at step 0: 8742.438634
Minibatch accuracy: 18.8%
Validation accuracy: 22.9%
Minibatch loss at step 500: 1647.674304
Minibatch accuracy: 82.0%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 1558.933030
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 1299.735235
Minibatch accuracy: 87.5%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 1180.242064
Minibatch accuracy: 90.6%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 1242.621355
Minibatch accuracy: 84.4%
Validation accuracy: 84.3%
Minibatch loss at step 3000: 1197.712323
Minibatch accuracy: 82.8%
Validation accuracy: 84.7%
Test accuracy: 91.7%


Highest accuracy acquired with a multi layer NN was 91.7%, lower than with a single hidden layer NN.